<a href="https://colab.research.google.com/github/hawc2/Text-Analysis-with-Python/blob/master/BERT_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resources

Tutorial on BERT OCR: 

https://www.statestitle.com/resource/using-nlp-bert-to-improve-ocr-accuracy/

Related: https://medium.com/towards-artificial-intelligence/using-tesseract-ocr-for-text-recognition-with-google-colab-1c4513b9d3e0

# Libraries


In [ ]:
!pip install pytesseract
!pip install pytorch-pretrained-bert
!pip3 install pyenchant

  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13945 sha256=386ea76f328ee4d6a31f7efd5e0a35b7957c6d76f9d1d266c1894b6440417c80
  Stored in directory: /root/.cache/pip/wheels/81/20/7e/1dd0daad1575d5260916bb1e9781246430647adaef4b3ca3b3
Successfully built pytesseract
     |████████████████████████████████| 133kB 5.6MB/s 
     |████████████████████████████████| 133kB 5.8MB/s 
     |████████████████████████████████| 71kB 4.7MB/s 
     |████████████████████████████████| 7.2MB 7.2MB/s 
ERROR: botocore 1.19.55 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 61kB 3.2MB/s 


In [ ]:
!sudo apt-get -y install libenchant1c2a
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 16 not upgraded.
Need to get 1,310 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libtext-iconv-perl amd64 1.7-5build6 [13.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libaspell15 amd64 0.60.7~20110707-4ubuntu0.1 [309 kB]
Get:3 http://a

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
from PIL import Image
import pytesseract
from pytesseract import image_to_string
import torch
from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM
import re
import nltk
from difflib import SequenceMatcher

In [ ]:
# this maybe fixed pyenchant install: https://github.com/pyenchant/pyenchant/issues/214
import enchant 
from enchant.checker import SpellChecker

# File Import

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 193696153.23_Page_134.png to 193696153.23_Page_134.png
User uploaded file "193696153.23_Page_134.png" with length 6484767 bytes


In [ ]:
filename = '19369615323_Page_134.png'
text = image_to_string(Image.open(filename))
text_original = str(text)
print (text_original)

124

Coleoptera
Pentamera.

ae

ENTOMOLOGY.

SrcTion 4TH, SIMPLICIMANI.

This section resembles the preceding so far as regards
the termination of the elytra ;! but the two anterior tarsi
only are dilated in the males, without however forming a
square or orbicular pallet; sometimes the first three ar-
ticles are obviously broader, and the succceding one 1s In
that case always much less than its antecedent ; sometimes
the latter and the two preceding ones are larger, almost
equal, and in the form of a reversed or triangular heart.
The joints of the four following tarsi are more slender and
elongated, almost cylindrical, or in the form of a length-
ened and reversed cone. | |

The Simplicimani of Latreille consist of genera belong-
ing to the tribe Carabiques L’eroniens of Dejean, with the
addition of Tetragonoderus, Dejean (Harpalici), Catasco-
pus, Kirby (Truncatipemnes), and a few others. _

Genus ZABrus, Clairville. The first three articles of
the anterior tarsi dilated in the males,

# Clean Text

In [ ]:
# cleanup text
rep = { '\n': ' ', '\\': ' ', '\"': '"', '-': ' ', '"': ' " ', 
        '"': ' " ', '"': ' " ', ',':' , ', '.':' . ', '!':' ! ', 
        '?':' ? ', "n't": " not" , "'ll": " will", '*':' * ', 
        '(': ' ( ', ')': ' ) ', "s'": "s '"}

In [ ]:
rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))
text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
def get_personslist(text):
    personslist=[]
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if isinstance(chunk, nltk.tree.Tree) and chunk.label() == 'PERSON':
                personslist.insert(0, (chunk.leaves()[0][0]))
    return list(set(personslist))
personslist = get_personslist(text)
ignorewords = personslist + ["!", ",", ".", "\"", "?", '(', ')', '*', "'"]


In [ ]:
# using enchant.checker.SpellChecker, identify incorrect words
# maybe replace SpellChecker
d = SpellChecker("en_US")
words = text.split()

incorrectwords = [w for w in words if not d.check(w) and w not in ignorewords]
# using enchant.checker.SpellChecker, get suggested replacements
suggestedwords = [d.suggest(w) for w in incorrectwords]

# replace incorrect words with [MASK]
for w in incorrectwords:
    text = text.replace(w, '[MASK]')
    text_original = text_original.replace(w, '[MASK]')

In [ ]:
print(text)

124  [MASK] [MASK] .   [MASK]  ENTOMOLOGY .   [MASK] 4TH ,  [MASK] .   This [MASK]c[MASK] re[MASK]mb[MASK] [MASK]e p[MASK]d[MASK] so f[MASK] as reg[MASK]ds [MASK]e termina[MASK] of [MASK]e [MASK] [MASK] !  but [MASK]e t[MASK] a[MASK]erior t[MASK]si only [MASK]e dilated in [MASK]e m[MASK]es ,  wi[MASK]out however form[MASK] a squ[MASK]e or orbicul[MASK] p[MASK]l[MASK][MASK] som[MASK]imes [MASK]e first [MASK]r[MASK] [MASK]  [MASK] [MASK]e obviously broa[MASK]r ,  and [MASK]e [MASK] one [MASK] In [MASK]at ca[MASK] [MASK]ways much [MASK]s [MASK]an its a[MASK]e[MASK][MASK][MASK] [MASK] som[MASK]imes [MASK]e latter and [MASK]e t[MASK] p[MASK]d[MASK] ones [MASK]e l[MASK]ger ,  [MASK]most equ[MASK] ,  and in [MASK]e form of a rever[MASK]d or triangul[MASK] he[MASK]t .  The joi[MASK]s of [MASK]e four follow[MASK] t[MASK]si [MASK]e more s[MASK]n[MASK]r and elongated ,  [MASK]most cylindric[MASK] ,  or in [MASK]e form of a [MASK]ng[MASK]  [MASK] and rever[MASK]d cone .  [MASK] [MASK]  The [MASK] 

# Load BERT Model

In [ ]:
# Load, train and predict using pre-trained model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_text = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
MASKIDS = [i for i, e in enumerate(tokenized_text) if e == '[MASK]']
# Create the segments tensors
segs = [i for i, e in enumerate(tokenized_text) if e == "."]
segments_ids=[]
prev=-1
for k, s in enumerate(segs):
    segments_ids = segments_ids + [k] * (s-prev)
    prev=s
segments_ids = segments_ids + [len(segs)] * (len(tokenized_text) - len(segments_ids))
segments_tensors = torch.tensor([segments_ids])
# prepare Torch inputs 
tokens_tensor = torch.tensor([indexed_tokens])
# Load pre-trained model
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
# Predict all tokens
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)

100%|██████████| 231508/231508 [00:00<00:00, 4184994.29B/s]
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (3699 > 512). Running this sequence through BERT will result in indexing errors
100%|██████████| 407873900/407873900 [00:06<00:00, 60055149.04B/s]


IndexError: ignored

With Peter's dataset: https://stackoverflow.com/questions/56010551/pytorch-embedding-index-out-of-range

# Refine BERT predictions

In [ ]:
#Predict words for mask using BERT; 
#refine prediction by matching with proposals from SpellChecker
def predict_word(text_original, predictions, maskids):
    pred_words=[]
    for i in range(len(MASKIDS)):
        preds = torch.topk(predictions[0, MASKIDS[i]], k=50) 
        indices = preds.indices.tolist()
        list1 = tokenizer.convert_ids_to_tokens(indices)
        list2 = suggestedwords[i]
        simmax=0
        predicted_token=''
        for word1 in list1:
            for word2 in list2:
                s = SequenceMatcher(None, word1, word2).ratio()
                if s is not None and s > simmax:
                    simmax = s
                    predicted_token = word1
        text_original = text_original.replace('[MASK]', predicted_token, 1)
    return text_original
text_original = predict_word(text_original, predictions, MASKIDS)
print (text_original)